In [ ]:
import tkinter as tk
from tkinter import filedialog
import random
import numpy as np
import cv2
import pandas as pd
import plotly.express as px

# Define the available shapes
SHAPES = ['star', 'triangle', 'circle', 'square', 'happy face']

# Define the available options for each setting
SIZE_OPTIONS = ['Small', 'Medium', 'Large', 'Random']
COMPLEXITY_OPTIONS = ['Low', 'Medium', 'High']
QUALITY_OPTIONS = ['Low', 'Medium', 'High', 'Random']
BACKGROUND_OPTIONS = ['Black', 'White', 'Random']

# Create a Tkinter window
window = tk.Tk()
window.title("Image Generation")
window.geometry('500x650')

# Create the fields and labels
default_resolution = 512

tk.Label(window, text="Number of Images:").grid(row=0, column=0)
num_images_entry = tk.Entry(window)
num_images_entry.insert(tk.END, '10')
num_images_entry.grid(row=0, column=1)

tk.Label(window, text="Resolution (x):").grid(row=1, column=0)
resolution_x_entry = tk.Entry(window)
resolution_x_entry.insert(tk.END, str(default_resolution))
resolution_x_entry.grid(row=1, column=1)

tk.Label(window, text="Resolution (y):").grid(row=2, column=0)
resolution_y_entry = tk.Entry(window)
resolution_y_entry.insert(tk.END, str(default_resolution))
resolution_y_entry.grid(row=2, column=1)

shape_vars = []
shape_checkboxes = []
for i, shape in enumerate(SHAPES):
    shape_var = tk.IntVar(value=1)
    shape_vars.append(shape_var)
    shape_checkboxes.append(tk.Checkbutton(window, text=shape, variable=shape_var))
    shape_checkboxes[i].grid(row=3 + i, column=0, sticky=tk.W)

primary_feature_label = tk.Label(window, text="Primary Feature:")
primary_feature_label.grid(row=8, column=0)
primary_feature_var = tk.StringVar(window)
primary_feature_var.set('happy face')
primary_feature_dropdown = tk.OptionMenu(window, primary_feature_var, *SHAPES)
primary_feature_dropdown.grid(row=8, column=1)

tk.Label(window, text="Shape Sizing:").grid(row=9, column=0)
size_var = tk.StringVar(window)
size_var.set(SIZE_OPTIONS[0])
size_dropdown = tk.OptionMenu(window, size_var, *SIZE_OPTIONS)
size_dropdown.grid(row=9, column=1)

tk.Label(window, text="Pattern Complexity:").grid(row=10, column=0)
complexity_var = tk.StringVar(window)
complexity_var.set(COMPLEXITY_OPTIONS[0])
complexity_dropdown = tk.OptionMenu(window, complexity_var, *COMPLEXITY_OPTIONS)
complexity_dropdown.grid(row=10, column=1)

tk.Label(window, text="Focus:").grid(row=11, column=0)
focus_var = tk.StringVar(window)
focus_var.set(QUALITY_OPTIONS[3])
focus_dropdown = tk.OptionMenu(window, focus_var, *QUALITY_OPTIONS)
focus_dropdown.grid(row=11, column=1)

tk.Label(window, text="Contrast:").grid(row=12, column=0)
contrast_var = tk.StringVar(window)
contrast_var.set(QUALITY_OPTIONS[3])
contrast_dropdown = tk.OptionMenu(window, contrast_var, *QUALITY_OPTIONS)
contrast_dropdown.grid(row=12, column=1)

tk.Label(window, text="Brightness:").grid(row=13, column=0)
brightness_var = tk.StringVar(window)
brightness_var.set(QUALITY_OPTIONS[3])
brightness_dropdown = tk.OptionMenu(window, brightness_var, *QUALITY_OPTIONS)
brightness_dropdown.grid(row=13, column=1)

tk.Label(window, text="Astigmatism:").grid(row=14, column=0)
astigmatism_var = tk.StringVar(window)
astigmatism_var.set(QUALITY_OPTIONS[3])
astigmatism_dropdown = tk.OptionMenu(window, astigmatism_var, *QUALITY_OPTIONS)
astigmatism_dropdown.grid(row=14, column=1)

tk.Label(window, text="Rotation:").grid(row=15, column=0)
rotation_var = tk.StringVar(window)
rotation_var.set(QUALITY_OPTIONS[3])
rotation_dropdown = tk.OptionMenu(window, rotation_var, *QUALITY_OPTIONS)
rotation_dropdown.grid(row=15, column=1)

default_save_location = "/mnt/c/Users/btorr/Desktop/Square_Script/100_Random_Shapes"


def browse_folder():
    save_location = filedialog.askdirectory()
    save_location_entry.delete(0, tk.END)
    save_location_entry.insert(tk.END, save_location)


tk.Label(window, text="Save Location:").grid(row=16, column=0)
save_location_entry = tk.Entry(window)
save_location_entry.insert(tk.END, default_save_location)
save_location_entry.grid(row=16, column=1)
browse_button = tk.Button(window, text="Browse", command=browse_folder)
browse_button.grid(row=16, column=2)

ideal_image_var = tk.IntVar(value=1)
ideal_image_checkbox = tk.Checkbutton(window, text="Ideal Image", variable=ideal_image_var)
ideal_image_checkbox.grid(row=17, column=0, sticky=tk.W)

random_location_var = tk.IntVar(value=0)
random_location_checkbox = tk.Checkbutton(window, text="Random Location for Ideal Image", variable=random_location_var)
random_location_checkbox.grid(row=18, column=0, sticky=tk.W)

tk.Label(window, text="Allow Shape Overlapping:").grid(row=19, column=0)
overlap_var = tk.IntVar(value=0)
overlap_checkbox = tk.Checkbutton(window, variable=overlap_var)
overlap_checkbox.grid(row=19, column=1, sticky=tk.W)

tk.Label(window, text="Background:").grid(row=20, column=0)
background_var = tk.StringVar(window)
background_var.set(BACKGROUND_OPTIONS[2])
background_dropdown = tk.OptionMenu(window, background_var, *BACKGROUND_OPTIONS)
background_dropdown.grid(row=20, column=1)


def generate_images():
    # Get the field values
    num_images = int(num_images_entry.get())
    resolution_x = int(resolution_x_entry.get())
    resolution_y = int(resolution_y_entry.get())
    selected_shapes = [SHAPES[i] for i, var in enumerate(shape_vars) if var.get() == 1]
    primary_feature = primary_feature_var.get()
    size_option = size_var.get()
    complexity_option = complexity_var.get()
    focus_option = focus_var.get()
    contrast_option = contrast_var.get()
    brightness_option = brightness_var.get()
    astigmatism_option = astigmatism_var.get()
    rotation_option = rotation_var.get()
    save_location = save_location_entry.get()
    allow_overlap = overlap_var.get() == 1
    background_option = background_var.get()

    # Generate ideal image
    if ideal_image_var.get() == 1:
        ideal_image = np.zeros((resolution_y, resolution_x), dtype=np.uint8)
        center_x = resolution_x // 2
        center_y = resolution_y // 2
        brightness_ideal = QUALITY_OPTIONS.index(brightness_option) - 1
        contrast_ideal = QUALITY_OPTIONS.index(contrast_option) - 1

        if primary_feature == 'star':
            cv2.drawMarker(ideal_image, (center_x, center_y), 255, cv2.MARKER_STAR, 20)
        elif primary_feature == 'triangle':
            points = np.array([(center_x - 50, center_y + 50),
                               (center_x + 50, center_y + 50),
                               (center_x, center_y - 50)], np.int32)
            cv2.fillPoly(ideal_image, [points], 255)
        elif primary_feature == 'circle':
            cv2.circle(ideal_image, (center_x, center_y), 50, 255, -1)
        elif primary_feature == 'square':
            cv2.rectangle(ideal_image, (center_x - 50, center_y - 50),
                          (center_x + 50, center_y + 50), 255, -1)
        elif primary_feature == 'happy face':
            cv2.ellipse(ideal_image, (center_x, center_y), (50, 50), 0, 0, 360, 255, -1)

        ideal_image = cv2.GaussianBlur(ideal_image, (3, 3), 0)
        ideal_image = cv2.addWeighted(ideal_image, 1 + brightness_ideal, 0, 0, contrast_ideal)

        if random_location_var.get() == 1:
            random_x = random.randint(0, resolution_x - 100)
            random_y = random.randint(0, resolution_y - 100)
            image = np.zeros((resolution_y, resolution_x), dtype=np.uint8)
            image[random_y:random_y + 100, random_x:random_x + 100] = ideal_image
            ideal_image = image

        cv2.imwrite(f"{save_location}/ideal_image.png", ideal_image)

    # Generate images
    for i in range(num_images):
        image = np.zeros((resolution_y, resolution_x), dtype=np.uint8)

        num_shapes = random.randint(10, 100)
        for _ in range(num_shapes):
            shape = random.choice(selected_shapes)
            size = random.uniform(0.05, 0.2) * min(resolution_x, resolution_y)
            position_x = random.randint(0, resolution_x)
            position_y = random.randint(0, resolution_y)

            if shape == 'star':
                cv2.drawMarker(image, (position_x, position_y), random.randint(0, 255), cv2.MARKER_STAR, int(size))
            elif shape == 'triangle':
                points = np.array([(position_x - int(size) // 2, position_y + int(size) // 2),
                                   (position_x + int(size) // 2, position_y + int(size) // 2),
                                   (position_x, position_y - int(size) // 2)], np.int32)
                cv2.fillPoly(image, [points], random.randint(0, 255))
            elif shape == 'circle':
                cv2.circle(image, (position_x, position_y), random.randint(0, int(size)), random.randint(0, 255), -1)
            elif shape == 'square':
                cv2.rectangle(image, (position_x - int(size) // 2, position_y - int(size) // 2),
                              (position_x + int(size) // 2, position_y + int(size) // 2),
                              random.randint(0, 255), -1)
            elif shape == 'happy face':
                cv2.ellipse(image, (position_x, position_y), (int(size) // 2, int(size) // 2), 0, 0, 360,
                            random.randint(0, 255), -1)

        size_random = SIZE_OPTIONS.index(size_option) - 1
        complexity_random = COMPLEXITY_OPTIONS.index(complexity_option) - 1
        focus_random = QUALITY_OPTIONS.index(focus_option) - 1
        contrast_random = QUALITY_OPTIONS.index(contrast_option) - 1
        brightness_random = QUALITY_OPTIONS.index(brightness_option) - 1
        astigmatism_random = QUALITY_OPTIONS.index(astigmatism_option) - 1
        rotation_random = QUALITY_OPTIONS.index(rotation_option) - 1

        image = cv2.GaussianBlur(image, (3, 3), int(focus_random))
        image = cv2.addWeighted(image, 1 + brightness_random, 0, 0, contrast_random)

        if allow_overlap:
            ideal_image = cv2.imread(f"{save_location}/ideal_image.png", 0)
            image = cv2.addWeighted(image, 0.5, ideal_image, 0.5, 0)

        image = cv2.rotate(image, random.choice([cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE]))

        cv2.imwrite(f"{save_location}/image_{i}.png", image)

    # Create a pandas DataFrame with image information
    data = {'Shape': [], 'Size': [], 'Complexity': [], 'Focus': [], 'Contrast': [], 'Brightness': [],
            'Astigmatism': [], 'Rotation': [], 'Background': []}

    for i in range(num_images):
        data['Shape'].append(', '.join(selected_shapes))
        data['Size'].append(size_option)
        data['Complexity'].append(complexity_option)
        data['Focus'].append(focus_option)
        data['Contrast'].append(contrast_option)
        data['Brightness'].append(brightness_option)
        data['Astigmatism'].append(astigmatism_option)
        data['Rotation'].append(rotation_option)
        data['Background'].append(background_option)

    df = pd.DataFrame(data)

    # Save the DataFrame as a CSV file
    df.to_csv(f"{save_location}/image_information.csv", index=False)

    # Generate a chart from the DataFrame
    fig = px.histogram(df, x='Shape')
    fig.show()


generate_button = tk.Button(window, text="Generate Images", command=generate_images)
generate_button.grid(row=21, column=0, columnspan=2)

window.mainloop()


In [ ]:
import random
import numpy as np
import cv2
import pandas as pd
import plotly.express as px
import tkinter as tk
from tkinter import filedialog

# Define the available shapes
SHAPES = ['star', 'triangle', 'circle', 'square', 'happy face']

# Define the available options for each setting
SIZE_OPTIONS = ['Small', 'Medium', 'Large', 'Random']
COMPLEXITY_OPTIONS = ['Low', 'Medium', 'High']
QUALITY_OPTIONS = ['Low', 'Medium', 'High', 'Random']

# Create a Tkinter window
window = tk.Tk()
window.title("Image Generation")
window.geometry('500x650')

# Create the fields and labels
default_resolution = 512
default_save_location = "/mnt/c/Users/btorr/Desktop/Python_Scripts"

tk.Label(window, text="Number of Images:").grid(row=0, column=0)
num_images_entry = tk.Entry(window)
num_images_entry.insert(tk.END, '10')
num_images_entry.grid(row=0, column=1)

tk.Label(window, text="Resolution (x):").grid(row=1, column=0)
resolution_x_entry = tk.Entry(window)
resolution_x_entry.insert(tk.END, str(default_resolution))
resolution_x_entry.grid(row=1, column=1)

tk.Label(window, text="Resolution (y):").grid(row=2, column=0)
resolution_y_entry = tk.Entry(window)
resolution_y_entry.insert(tk.END, str(default_resolution))
resolution_y_entry.grid(row=2, column=1)

shape_vars = []
shape_checkboxes = []
for i, shape in enumerate(SHAPES):
    shape_var = tk.IntVar(value=1)
    shape_vars.append(shape_var)
    shape_checkboxes.append(tk.Checkbutton(window, text=shape, variable=shape_var))
    shape_checkboxes[i].grid(row=3 + i, column=0, sticky=tk.W)

primary_feature_label = tk.Label(window, text="Primary Feature:")
primary_feature_label.grid(row=8, column=0)
primary_feature_var = tk.StringVar(window)
primary_feature_var.set('happy face')
primary_feature_dropdown = tk.OptionMenu(window, primary_feature_var, *SHAPES)
primary_feature_dropdown.grid(row=8, column=1)

tk.Label(window, text="Shape Sizing:").grid(row=9, column=0)
size_var = tk.StringVar(window)
size_var.set(SIZE_OPTIONS[0])
size_dropdown = tk.OptionMenu(window, size_var, *SIZE_OPTIONS)
size_dropdown.grid(row=9, column=1)

tk.Label(window, text="Pattern Complexity:").grid(row=10, column=0)
complexity_var = tk.StringVar(window)
complexity_var.set(COMPLEXITY_OPTIONS[0])
complexity_dropdown = tk.OptionMenu(window, complexity_var, *COMPLEXITY_OPTIONS)
complexity_dropdown.grid(row=10, column=1)

tk.Label(window, text="Focus:").grid(row=11, column=0)
focus_var = tk.StringVar(window)
focus_var.set(QUALITY_OPTIONS[3])
focus_dropdown = tk.OptionMenu(window, focus_var, *QUALITY_OPTIONS)
focus_dropdown.grid(row=11, column=1)

tk.Label(window, text="Contrast:").grid(row=12, column=0)
contrast_var = tk.StringVar(window)
contrast_var.set(QUALITY_OPTIONS[3])
contrast_dropdown = tk.OptionMenu(window, contrast_var, *QUALITY_OPTIONS)
contrast_dropdown.grid(row=12, column=1)

tk.Label(window, text="Brightness:").grid(row=13, column=0)
brightness_var = tk.StringVar(window)
brightness_var.set(QUALITY_OPTIONS[3])
brightness_dropdown = tk.OptionMenu(window, brightness_var, *QUALITY_OPTIONS)
brightness_dropdown.grid(row=13, column=1)

tk.Label(window, text="Astigmatism:").grid(row=14, column=0)
astigmatism_var = tk.StringVar(window)
astigmatism_var.set(QUALITY_OPTIONS[3])
astigmatism_dropdown = tk.OptionMenu(window, astigmatism_var, *QUALITY_OPTIONS)
astigmatism_dropdown.grid(row=14, column=1)

tk.Label(window, text="Rotation:").grid(row=15, column=0)
rotation_var = tk.StringVar(window)
rotation_var.set(QUALITY_OPTIONS[3])
rotation_dropdown = tk.OptionMenu(window, rotation_var, *QUALITY_OPTIONS)
rotation_dropdown.grid(row=15, column=1)

def browse_folder():
    save_location = filedialog.askdirectory()
    save_location_entry.delete(0, tk.END)
    save_location_entry.insert(tk.END, save_location)

tk.Label(window, text="Save Location:").grid(row=16, column=0)
save_location_entry = tk.Entry(window)
save_location_entry.insert(tk.END, default_save_location)
save_location_entry.grid(row=16, column=1)
browse_button = tk.Button(window, text="Browse", command=browse_folder)
browse_button.grid(row=16, column=2)

ideal_image_var = tk.IntVar(value=1)
ideal_image_checkbox = tk.Checkbutton(window, text="Ideal Image", variable=ideal_image_var)
ideal_image_checkbox.grid(row=17, column=0, sticky=tk.W)

random_location_var = tk.IntVar(value=0)
random_location_checkbox = tk.Checkbutton(window, text="Random Location for Ideal Image", variable=random_location_var)
random_location_checkbox.grid(row=18, column=0, sticky=tk.W)

tk.Label(window, text="Allow Shape Overlapping:").grid(row=19, column=0)
overlap_var = tk.IntVar(value=0)
overlap_checkbox = tk.Checkbutton(window, variable=overlap_var)
overlap_checkbox.grid(row=19, column=1, sticky=tk.W)

def generate_images():
    # Get the field values
    num_images = int(num_images_entry.get())
    resolution_x = int(resolution_x_entry.get())
    resolution_y = int(resolution_y_entry.get())
    selected_shapes = [SHAPES[i] for i, var in enumerate(shape_vars) if var.get() == 1]
    primary_feature = primary_feature_var.get()
    size_option = size_var.get()
    complexity_option = complexity_var.get()
    focus_option = focus_var.get()
    contrast_option = contrast_var.get()
    brightness_option = brightness_var.get()
    astigmatism_option = astigmatism_var.get()
    rotation_option = rotation_var.get()
    save_location = save_location_entry.get()
    allow_overlap = overlap_var.get() == 1

    # Generate ideal image
    if ideal_image_var.get() == 1:
        ideal_image = np.zeros((resolution_y, resolution_x, 3), dtype=np.uint8)
        center_x = resolution_x // 2
        center_y = resolution_y // 2

        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        if primary_feature == 'star':
            cv2.drawMarker(ideal_image, (center_x, center_y), color, cv2.MARKER_STAR, 20)
        elif primary_feature == 'triangle':
            points = np.array([(center_x - 50, center_y + 50),
                               (center_x + 50, center_y + 50),
                               (center_x, center_y - 50)], np.int32)
            cv2.fillPoly(ideal_image, [points], color)
        elif primary_feature == 'circle':
            cv2.circle(ideal_image, (center_x, center_y), 50, color, -1)
        elif primary_feature == 'square':
            cv2.rectangle(ideal_image, (center_x - 50, center_y - 50),
                          (center_x + 50, center_y + 50), color, -1)
        elif primary_feature == 'happy face':
            cv2.ellipse(ideal_image, (center_x, center_y), (50, 50), 0, 0, 360, color, -1)

        ideal_image = cv2.GaussianBlur(ideal_image, (3, 3), 0)
        ideal_image = cv2.addWeighted(ideal_image, 1 + int(brightness_option), 0, 0, int(contrast_option))

        if random_location_var.get() == 1:
            random_x = random.randint(0, resolution_x - 100)
            random_y = random.randint(0, resolution_y - 100)
            image = np.zeros((resolution_y, resolution_x, 3), dtype=np.uint8)
            image[random_y:random_y + 100, random_x:random_x + 100] = ideal_image
            ideal_image = image

        cv2.imwrite(f"{save_location}/ideal_image.png", ideal_image)

    # Generate images
    for i in range(num_images):
        image = np.zeros((resolution_y, resolution_x, 3), dtype=np.uint8)

        num_shapes = random.randint(10, 100)
        for _ in range(num_shapes):
            shape = random.choice(selected_shapes)
            size = random.uniform(0.05, 0.2) * min(resolution_x, resolution_y)
            position_x = random.randint(0, resolution_x)
            position_y = random.randint(0, resolution_y)
            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

            if shape == 'star':
                cv2.drawMarker(image, (position_x, position_y), color, cv2.MARKER_STAR, int(size))
            elif shape == 'triangle':
                points = np.array([(position_x - int(size) // 2, position_y + int(size) // 2),
                                   (position_x + int(size) // 2, position_y + int(size) // 2),
                                   (position_x, position_y - int(size) // 2)], np.int32)
                cv2.fillPoly(image, [points], color)
            elif shape == 'circle':
                cv2.circle(image, (position_x, position_y), random.randint(0, int(size)), color, -1)
            elif shape == 'square':
                cv2.rectangle(image, (position_x - int(size) // 2, position_y - int(size) // 2),
                              (position_x + int(size) // 2, position_y + int(size) // 2),
                              color, -1)
            elif shape == 'happy face':
                cv2.ellipse(image, (position_x, position_y), (int(size) // 2, int(size) // 2), 0, 0, 360,
                            color, -1)

        size_random = SIZE_OPTIONS.index(size_option) - 1
        complexity_random = COMPLEXITY_OPTIONS.index(complexity_option) - 1
        focus_random = QUALITY_OPTIONS.index(focus_option) - 1
        contrast_random = QUALITY_OPTIONS.index(contrast_option) - 1
        brightness_random = QUALITY_OPTIONS.index(brightness_option) - 1
        astigmatism_random = QUALITY_OPTIONS.index(astigmatism_option) - 1
        rotation_random = QUALITY_OPTIONS.index(rotation_option) - 1

        image = cv2.GaussianBlur(image, (3, 3), int(focus_random))
        image = cv2.addWeighted(image, 1 + brightness_random, 0, 0, contrast_random)

        if allow_overlap:
            ideal_image = cv2.imread(f"{save_location}/ideal_image.png", 1)
            image = cv2.addWeighted(image, 0.5, ideal_image, 0.5, 0)

        image = cv2.rotate(image, random.choice([cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE]))

        cv2.imwrite(f"{save_location}/image_{i}.png", image)

    # Create a pandas DataFrame with image information
    data = {'Shape': [], 'Size': [], 'Complexity': [], 'Focus': [], 'Contrast': [], 'Brightness': [],
            'Astigmatism': [], 'Rotation': [], 'Background': []}

    for i in range(num_images):
        data['Shape'].append(', '.join(selected_shapes))
        data['Size'].append(size_option)
        data['Complexity'].append(complexity_option)
        data['Focus'].append(focus_option)
        data['Contrast'].append(contrast_option)
        data['Brightness'].append(brightness_option)
        data['Astigmatism'].append(astigmatism_option)
        data['Rotation'].append(rotation_option)
        data['Background'].append("Random")

    df = pd.DataFrame(data)

    # Save the DataFrame as a CSV file
    df.to_csv(f"{save_location}/image_information.csv", index=False)

    # Generate a chart from the DataFrame
    fig = px.histogram(df, x='Shape')
    fig.show()


generate_button = tk.Button(window, text="Generate Images", command=generate_images)
generate_button.grid(row=21, column=0, columnspan=2)

window.mainloop()
